In [1]:
import sys
!{sys.executable} -m pip install missingno
!{sys.executable} -m pip install stringcase


You should consider upgrading via the '/dataquest/system/env/python3/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/dataquest/system/env/python3/bin/python3 -m pip install --upgrade pip' command.


In [2]:
# create a new class which makes it possible to format text in colour or bold etc.

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [3]:
import pandas as pd
import numpy as np
import missingno as msno
import datetime as dt
from stringcase import snakecase

## Analyzing Used Car Listings on eBay Kleinanzeigen
This is an analysis of used car listings on eBay Kleinanzeigen, a classifieds section of the German eBay website.

During this analysis data will be cleaned and explored. After cleaning and exploration the top ten car brands will be analysed in more detail (mean price and mean mileage). Lastly the most popular models for each brand in dataset will be determined and the effect of gearbox model and unrepaired damage on the price will be investigated.

The dataset was originally scraped and uploaded to Kaggle. The version of the dataset that is used for this analysis is a sample of 50,000 data points that was prepared by Dataquest including simulating a less-cleaned version of the data.

## Short exploration

In [4]:
autos = pd.read_csv('autos.csv',encoding='Latin-1')
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [5]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

Column description can be seen below:


|Column|	Description|
|:---|:----|
|dateCrawled|	When this ad was first crawled, all field-values are taken from this date|
|name	|Name of the car|
|seller	|Whether the seller is private or a dealer|
|offerType	|The type of listing|
|price	|The price on the ad to sell the car|
|abtest	|Whether the listing is included in an A/B test|
|vehicleType	|The vehicle Type|
|yearOfRegistration	|The year in which the car was first registered|
|gearbox	|The transmission type|
|powerPS	|The power of the car in PS|
|model	|The car model name|
|odometer	|How many kilometers the car has driven|
|monthOfRegistration	|The month in which the car was first registered|
|fuelType	|What type of fuel the car uses|
|brand	|The brand of the car|
|notRepairedDamage	|If the car has a damage which is not yet repaired|
|dateCreated	|The date on which the eBay listing was created|
|nrOfPictures	|The number of pictures in the ad|
|postalCode	|The postal code for the location of the vehicle|
|lastSeenOnline| the crawler saw this ad last online|

From two last cells we can find out:

The dataset contains 20 columns, most of which are strings.

Some columns have null values, but none have more than ~20% null values.

The column names use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscores.

Let's convert the column names from camelcase to snakecase and reword some of the column names based on the data dictionary to be more descriptive.

In [6]:
#turning camelcase columns into snakecase columns
snakecase_col = []    
    
for col in autos.columns:
    col = snakecase(col)
    snakecase_col.append(col)
    
autos.columns = snakecase_col

#renaming some columns by using a for loop with old and new values
rename_columns = [
    ['year_of_registration','registration_year'],
    ['month_of_registration','registration_month'],
    ['not_repaired_damage','unrepaired_damage'],
    ['date_created','ad_created'],
    ['power_p_s','power_PS']
]

for old_col, new_col in rename_columns:
    autos.rename({old_col: new_col},axis=1,inplace=True)

Now let's do some basic data exploration to determine what other cleaning tasks need to be done. Initially we will look for:

Text columns where all or almost all values are the same. These can often be dropped as they don't have useful information for analysis.
Examples of numeric data stored as text which can be cleaned and converted.

In [7]:
autos.describe(include = 'all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-02 15:49:30,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN



It can be seen that columns "seller" and "offer_type" consist of almost all the same values. Therefore, these columns are deemed unuseful for further analysis and will be dropped from the dataframe.


In [8]:
#remove columns that provide unuseful data
autos.drop(['seller','offer_type'],axis=1,inplace=True)

Also, the "price" and "odometer" columns seem to contain numeric data but are currently stored as object data. Let's transform them into float and change the col names.


In [9]:
# transforming price col into float
autos.price = (autos.price
               .str.replace('$' , '')
               .str.replace(',' , '')
               .astype(float)
              )


# transforming odometer col into float
autos.odometer = (autos.odometer
               .str.replace('km' , '')
               .str.replace(',' , '')
               .astype(float)
              )

autos.rename({'odometer': 'odometer_km'},axis=1,inplace=True)

In [10]:
def explore_series(dataframe, column): #define a function to give detailed info about a column
    

    
          
    # length of .unique()
    template = (color.BOLD + "Number of unique values in column " 
                + color.RED + column + color.END 
                + color.BOLD + " is " 
                + color.RED + "{:,} " + color.END)
    print(template.format(len(dataframe[column].unique())))
    print('\n')
    
    # .describe()
    print(color.BOLD + "Descriptive statistics of column " 
          + color.RED + column + color.END)
    print(dataframe[column].describe())
    print('\n')
    
    #  highest prices ()
    print(color.BOLD + 'highest values and their counts for column '
          + color.RED + column + color.END)
    print(autos[column].value_counts().sort_index(ascending=False).head(25))
    print('\n')
    
    
    # 25 lowest prices
    print(color.BOLD + 'lowest values  and their counts for column '
          + color.RED + column + color.END)
    print(autos[column].value_counts().sort_index(ascending=True).head(25))
    print('\n')

In [11]:
explore_series(autos,"price")

Number of unique values in column price is 2,357 


Descriptive statistics of column price
count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64


highest values and their counts for column price
99999999.0    1
27322222.0    1
12345678.0    3
11111111.0    2
10000000.0    1
3890000.0     1
1300000.0     1
1234566.0     1
999999.0      2
999990.0      1
350000.0      1
345000.0      1
299000.0      1
295000.0      1
265000.0      1
259000.0      1
250000.0      1
220000.0      1
198000.0      1
197000.0      1
194000.0      1
190000.0      1
180000.0      1
175000.0      1
169999.0      1
Name: price, dtype: int64


lowest values  and their counts for column price
0.0     1421
1.0      156
2.0        3
3.0        1
5.0        2
8.0        1
9.0        1
10.0       7
11.0       2
12.0       3
13.0       2
14.0       1
15.0       2
17.0    

A mean of nearly ten thousand dollars for used cars of all types seems realistic.
The mean is only a fraction of the standard deviation (which is 481,000$).
This suggests that there are outliers in the dataset.

The lowest price is zero, which has 1421 occurences in the dataset. Anything below 500$ (which seems like a realistic lower bound for used car prices) will be removed from the dataset.

There are also outliers at the top. When investigating the unique values with their respective counts the starting price increases slowly until 350,000.
Onwards the price increases rapidly. Occurences with a starting price higher than 350,000 will be investigated further.

In [12]:
price_above_350000 = autos[autos['price']>350000]

template = (color.BOLD + "There are " 
            + color.RED + "{}" + color.END 
            + color.BOLD + " instances with a starting price of 350,000 or higher" + color.END)

print(template.format(len(price_above_350000)))
    
price_above_350000

There are 14 instances with a starting price of 350,000 or higher


,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
514,2016-03-17 09:53:08,Ford_Focus_Turnier_1.6_16V_Style,999999.0,test,kombi,2009,manuell,101,focus,125000.0,4,benzin,ford,nein,2016-03-17 00:00:00,0,12205,2016-04-06 07:17:35
2897,2016-03-12 21:50:57,Escort_MK_1_Hundeknochen_zum_umbauen_auf_RS_2000,11111111.0,test,limousine,1973,manuell,48,escort,50000.0,3,benzin,ford,nein,2016-03-12 00:00:00,0,94469,2016-03-12 22:45:27
7814,2016-04-04 11:53:31,Ferrari_F40,1300000.0,control,coupe,1992,NaN,0,NaN,50000.0,12,NaN,sonstige_autos,nein,2016-04-04 00:00:00,0,60598,2016-04-05 11:34:11
11137,2016-03-29 23:52:57,suche_maserati_3200_gt_Zustand_unwichtig_laufe...,10000000.0,control,coupe,1960,manuell,368,NaN,100000.0,1,benzin,sonstige_autos,nein,2016-03-29 00:00:00,0,73033,2016-04-06 21:18:11
22947,2016-03-22 12:54:19,Bmw_530d_zum_ausschlachten,1234566.0,control,kombi,1999,automatik,190,NaN,150000.0,2,diesel,bmw,NaN,2016-03-22 00:00:00,0,17454,2016-04-02 03:17:32
24384,2016-03-21 13:57:51,Schlachte_Golf_3_gt_tdi,11111111.0,test,NaN,1995,NaN,0,NaN,150000.0,0,NaN,volkswagen,NaN,2016-03-21 00:00:00,0,18519,2016-03-21 14:40:18
27371,2016-03-09 15:45:47,Fiat_Punto,12345678.0,control,NaN,2017,NaN,95,punto,150000.0,0,NaN,fiat,NaN,2016-03-09 00:00:00,0,96110,2016-03-09 15:45:47
37585,2016-03-29 11:38:54,Volkswagen_Jetta_GT,999990.0,test,limousine,1985,manuell,111,jetta,150000.0,12,benzin,volkswagen,ja,2016-03-29 00:00:00,0,50997,2016-03-29 11:38:54
39377,2016-03-08 23:53:51,Tausche_volvo_v40_gegen_van,12345678.0,control,NaN,2018,manuell,95,v40,150000.0,6,NaN,volvo,nein,2016-03-08 00:00:00,0,14542,2016-04-06 23:17:31
39705,2016-03-22 14:58:27,Tausch_gegen_gleichwertiges,99999999.0,control,limousine,1999,automatik,224,s_klasse,150000.0,9,benzin,mercedes_benz,NaN,2016-03-22 00:00:00,0,73525,2016-04-06 05:15:30


There are 14 occurences with a price higher than 350,000.\ Since most of these cars are normal family cars (with brands like Ford, BMW, Fiat, Volkswagen) these prices are unrealistic.
The 14 instances will be removed from the dataset.

In [13]:
autos = autos.loc[autos.price.between(500,350000)]

explore_series(autos,'price')


Number of unique values in column price is 2,208 


Descriptive statistics of column price
count     45097.000000
mean       6320.659600
std        9261.841444
min         500.000000
25%        1500.000000
50%        3500.000000
75%        7900.000000
max      350000.000000
Name: price, dtype: float64


highest values and their counts for column price
350000.0    1
345000.0    1
299000.0    1
295000.0    1
265000.0    1
259000.0    1
250000.0    1
220000.0    1
198000.0    1
197000.0    1
194000.0    1
190000.0    1
180000.0    1
175000.0    1
169999.0    1
169000.0    1
163991.0    1
163500.0    1
155000.0    1
151990.0    1
145000.0    1
139997.0    1
137999.0    1
135000.0    1
130000.0    1
Name: price, dtype: int64


lowest values  and their counts for column price
500.0    781
501.0      1
510.0      2
517.0      1
520.0      8
525.0      4
530.0      8
540.0      2
549.0     13
550.0    356
554.0      1
555.0     33
560.0      4
566.0      1
570.0      7
575.0      5
578.0      

After removal there are 45,097 instances left in the dataset.
Removing the outliers at the top has had its impact on the mean starting price.
This has decreased from 9,840 to 6,320.
The standard deviation has decreased drastically as it is now less than 9,500.

The "odometer_km" will be explored next.

In [14]:
explore_series(autos, 'odometer_km')

Number of unique values in column odometer_km is 13 


Descriptive statistics of column odometer_km
count     45097.000000
mean     125293.035013
std       39622.744927
min        5000.000000
25%      100000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64


highest values and their counts for column odometer_km
150000.0    28698
125000.0     4838
100000.0     2031
90000.0      1676
80000.0      1385
70000.0      1189
60000.0      1131
50000.0       996
40000.0       808
30000.0       765
20000.0       727
10000.0       237
5000.0        616
Name: odometer_km, dtype: int64


lowest values  and their counts for column odometer_km
5000.0        616
10000.0       237
20000.0       727
30000.0       765
40000.0       808
50000.0       996
60000.0      1131
70000.0      1189
80000.0      1385
90000.0      1676
100000.0     2031
125000.0     4838
150000.0    28698
Name: odometer_km, dtype: int64




There are only 13 different values of mileages in the dataset. This suggests that people have to use a preset of values when creating an auction on eBay.

The lowest value is 5,000 and the highest is 150,000 which are both plausible values for mileage on a used car.
Therefore there will be no correction on outliers for this column.

The mean is more than 120,000 km which suggests that most cars sold through eBay are likely to be a couple years old.
50% of the dataset has a mileage of 150,000 (or more).

This concludes the exploration of the columns "price" and "odometer_km".

## Exploration of date columns

There are five columns containing date information.

|Column|Description|
|:-----|:------|
|date_crawled|When this ad was first crawled, all field-values are taken from this date|
|registration_year|The year in which the car was first registered|
|registration_month|The month in which the car was first registered
|
|ad_created|The date on which the eBay listing was created
|
|last_seen|When the crawler saw this ad last online
|

In order to explore the date columns more easily date_crawled, ad_created and last_seen have to be reformatted into a date type.
Once this has been done all columns will be explored shortly.

In [15]:
#creating a function to reformat the columns with string format into date format.
def formatDate(dataset,column):
    newformat = []
    for element in dataset[column]:
        element = dt.datetime.strptime(element, "%Y-%m-%d %H:%M:%S")
        element = element.date()
        newformat.append(element)
    dataset[column] = newformat
                   
formatDate(autos,'date_crawled')
formatDate(autos,'ad_created')
formatDate(autos,'last_seen')

### date_crawled

In [16]:
explore_series(autos,'date_crawled')

Number of unique values in column date_crawled is 34 


Descriptive statistics of column date_crawled
count          45097
unique            34
top       2016-04-03
freq            1751
Name: date_crawled, dtype: object


highest values and their counts for column date_crawled
2016-04-07      61
2016-04-06     143
2016-04-05     594
2016-04-04    1651
2016-04-03    1751
2016-04-02    1613
2016-04-01    1529
2016-03-31    1428
2016-03-30    1503
2016-03-29    1500
2016-03-28    1571
2016-03-27    1406
2016-03-26    1472
2016-03-25    1402
2016-03-24    1307
2016-03-23    1461
2016-03-22    1489
2016-03-21    1702
2016-03-20    1717
2016-03-19    1567
2016-03-18     581
2016-03-17    1405
2016-03-16    1324
2016-03-15    1534
2016-03-14    1637
Name: date_crawled, dtype: int64


lowest values  and their counts for column date_crawled
2016-03-05    1153
2016-03-06     637
2016-03-07    1632
2016-03-08    1496
2016-03-09    1484
2016-03-10    1475
2016-03-11    1489
2016-03-12    1683
2016

Looking at the data above it seems like the period over which the data has been crawled covers roughly one month (March-April 2016).
The distribution is more or less uniform.

### ad_created

In [17]:
explore_series(autos,'ad_created')

Number of unique values in column ad_created is 76 


Descriptive statistics of column ad_created
count          45097
unique            76
top       2016-04-03
freq            1761
Name: ad_created, dtype: object


highest values and their counts for column ad_created
2016-04-07      54
2016-04-06     147
2016-04-05     537
2016-04-04    1668
2016-04-03    1761
2016-04-02    1599
2016-04-01    1526
2016-03-31    1431
2016-03-30    1497
2016-03-29    1499
2016-03-28    1573
2016-03-27    1404
2016-03-26    1472
2016-03-25    1409
2016-03-24    1306
2016-03-23    1453
2016-03-22    1478
2016-03-21    1712
2016-03-20    1723
2016-03-19    1516
2016-03-18     609
2016-03-17    1390
2016-03-16    1346
2016-03-15    1524
2016-03-14    1573
Name: ad_created, dtype: int64


lowest values  and their counts for column ad_created
2015-06-11    1
2015-08-10    1
2015-09-09    1
2015-11-10    1
2015-12-05    1
2015-12-30    1
2016-01-03    1
2016-01-07    1
2016-01-10    2
2016-01-13    1
2016-01-

The dates ads were created range from June 2015 until April of 2016.

### last_seen

In [18]:
explore_series(autos, 'last_seen')

Number of unique values in column last_seen is 34 


Descriptive statistics of column last_seen
count          45097
unique            34
top       2016-04-06
freq           10161
Name: last_seen, dtype: object


highest values and their counts for column last_seen
2016-04-07     6050
2016-04-06    10161
2016-04-05     5710
2016-04-04     1096
2016-04-03     1125
2016-04-02     1122
2016-04-01     1031
2016-03-31     1057
2016-03-30     1089
2016-03-29      963
2016-03-28      926
2016-03-27      697
2016-03-26      743
2016-03-25      838
2016-03-24      881
2016-03-23      830
2016-03-22      958
2016-03-21      932
2016-03-20      921
2016-03-19      695
2016-03-18      334
2016-03-17     1248
2016-03-16      729
2016-03-15      707
2016-03-14      554
Name: last_seen, dtype: int64


lowest values  and their counts for column last_seen
2016-03-05      49
2016-03-06     188
2016-03-07     235
2016-03-08     316
2016-03-09     427
2016-03-10     464
2016-03-11     543
2016-03-12    10

The last three days contain a disproportionate amount of 'last seen' values.\ Given that these are 6-10x the values from the previous days, it's unlikely that there was a massive spike in sales.
It's more likely that these values are to do with the crawling period ending and don't indicate car sales.

### registration_year

In [19]:
explore_series(autos, 'registration_year')

Number of unique values in column registration_year is 93 


Descriptive statistics of column registration_year
count    45097.000000
mean      2005.064173
std         89.652017
min       1000.000000
25%       2000.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64


highest values and their counts for column registration_year
9999       3
9000       1
8888       1
6200       1
5911       1
5000       2
4800       1
4500       1
4100       1
2800       1
2019       1
2018     462
2017    1296
2016     950
2015     364
2014     652
2013     797
2012    1307
2011    1617
2010    1586
2009    2078
2008    2206
2007    2270
2006    2666
2005    2864
Name: registration_year, dtype: int64


lowest values  and their counts for column registration_year
1000     1
1001     1
1910     2
1927     1
1929     1
1931     1
1934     2
1937     4
1938     1
1939     1
1941     2
1943     1
1948     1
1950     1
1951     2
1952     1
1953   

Both the minimum as maximum value of registration year seems strange.

The lowest registration year is 1,000 which must be incorrect as cars only started appearing in the late 1800's.
Due to this all occurences with a registration year before 1885 (first patented practical automobile) will be removed.

All registration years after 2016 must be incorrect as ads were created in 2015 & 2016.
These will be removed from the dataset as well.

In [20]:
autos = autos[autos['registration_year'].between(1885,2016)]
explore_series(autos, 'registration_year')

Number of unique values in column registration_year is 78 


Descriptive statistics of column registration_year
count    43323.000000
mean      2003.227270
std          7.075229
min       1910.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       2016.000000
Name: registration_year, dtype: float64


highest values and their counts for column registration_year
2016     950
2015     364
2014     652
2013     797
2012    1307
2011    1617
2010    1586
2009    2078
2008    2206
2007    2270
2006    2666
2005    2864
2004    2683
2003    2657
2002    2419
2001    2520
2000    2715
1999    2572
1998    2026
1997    1512
1996    1072
1995     860
1994     500
1993     340
1992     305
Name: registration_year, dtype: int64


lowest values  and their counts for column registration_year
1910     2
1927     1
1929     1
1931     1
1934     2
1937     4
1938     1
1939     1
1941     2
1943     1
1948     1
1950     1
1951     2
1952     1
1953     1
1954     2
1955   

There is still a large variety in registration years of the cars.\ The mean of 2002 with a small standard deviation indicate that most cars are approximately between 7 and 21 years old.

### Exploration of variations between car brands¶
When working with data on cars, it's natural to explore variations across different car brands.
Aggregation can be used to understand the brand column.

In [21]:
explore_series(autos, 'brand')

Number of unique values in column brand is 40 


Descriptive statistics of column brand
count          43323
unique            40
top       volkswagen
freq            9180
Name: brand, dtype: object


highest values and their counts for column brand
volvo              406
volkswagen        9180
trabant             48
toyota             581
suzuki             256
subaru              86
sonstige_autos     420
smart              655
skoda              743
seat               775
saab                72
rover               54
renault           1902
porsche            278
peugeot           1272
opel              4347
nissan             650
mitsubishi         335
mini               406
mercedes_benz     4430
mazda              650
land_rover          98
lancia              43
lada                26
kia                318
Name: brand, dtype: int64


lowest values  and their counts for column brand
alfa_romeo        287
audi             3921
bmw              4983
chevrolet         261
chrysler  

There is a large variety of car brands in the dataset and data is not uniformly distributed.
For further analysis the dataset will be limited to the top 10 brands in terms of number of ads.

In [22]:
top_10_brands = (autos['brand']
                 .value_counts(normalize=True,dropna=False)
                 .head(10)
                 .index)

autos = autos[autos.brand.isin(top_10_brands)]

In [23]:
explore_series(autos, 'brand')

Number of unique values in column brand is 10 


Descriptive statistics of column brand
count          34652
unique            10
top       volkswagen
freq            9180
Name: brand, dtype: object


highest values and their counts for column brand
volkswagen       9180
seat              775
renault          1902
peugeot          1272
opel             4347
mercedes_benz    4430
ford             2823
fiat             1019
bmw              4983
audi             3921
Name: brand, dtype: int64


lowest values  and their counts for column brand
audi             3921
bmw              4983
fiat             1019
ford             2823
mercedes_benz    4430
opel             4347
peugeot          1272
renault          1902
seat              775
volkswagen       9180
Name: brand, dtype: int64




There are still more then 34,000 entries left after limiting the dataset to the top 10 brands. All brands left are European, except for Ford. Out of those 10, 5 are German which can be explained by the dataset being from the German section of eBay.

Now that the dataset contains only the top 10 brands it might be interesting to check whether there are differences in mean price between those brands.

In [24]:
mean_price_per_brand = {}

for brand in top_10_brands:
    mean_price = autos.loc[autos['brand'] == brand,'price'].mean()
    mean_price_per_brand[brand] = int(round(mean_price,0))

mean_price_per_brand = pd.Series(mean_price_per_brand, name='mean_price')

In [25]:
mean_price_per_brand.sort_values(ascending=False)

audi             9614
mercedes_benz    8767
bmw              8582
volkswagen       5784
seat             4811
ford             4292
opel             3394
peugeot          3361
fiat             3256
renault          2819
Name: mean_price, dtype: int64

Brands can be divided into three price categories:

Audi, Mercedes Benz and BMW being expensive (above €8,000)
Volkswagen and Seat being average priced (between €4,000 and €8,000)
Ford, Peugeut, Opel, Fiat, and Renault being cheap (below €4,000)
Normally a car will become less valuable when the mileage of the car becomes higher.
It is interesting to see whether this principle can be found in the dataset.
For that the mean mileage per brand needs to be calculated.

In [26]:
mean_mileage_per_brand = {}

for brand in top_10_brands:
    mean_mileage = autos.loc[autos['brand'] == brand,'odometer_km'].mean()
    mean_mileage_per_brand[brand] = int(round(mean_mileage,0))
    
mean_mileage_per_brand = pd.Series(mean_mileage_per_brand,name='mean_mileage')

mean_mileage_per_brand 

volkswagen       128235
bmw              132866
mercedes_benz    131019
opel             128012
audi             128910
ford             123495
renault          126351
peugeot          126073
fiat             114416
seat             120058
Name: mean_mileage, dtype: int64

In [27]:
price_mileage_per_brand = pd.concat([mean_price_per_brand,mean_mileage_per_brand],axis=1)

price_mileage_per_brand

,mean_price,mean_mileage
volkswagen,5784,128235
bmw,8582,132866
mercedes_benz,8767,131019
opel,3394,128012
audi,9614,128910
ford,4292,123495
renault,2819,126351
peugeot,3361,126073
fiat,3256,114416
seat,4811,120058


Above both mean price and mean mileage can be seen. It is however not possible to conclude whether higher mileage is affecting the price.
This is due to the fact that within a brand there are a lot of other variables affecting price (such as car type, engine type, registration year etc).
In order to confirm whether mileage affects the price a slice of the dataset is necessary where all those variables are kept the same as much as possible.

In [28]:
unique_brands = autos.brand.unique()

model_brand = {}
model_brand_count = {}
for ub in unique_brands:
    m = autos.loc[autos.brand == ub , 'model'].value_counts().index[0]
    c = autos.loc[autos.brand == ub , 'model'].value_counts()[0]
    model_brand[ub] = m
    model_brand_count[ub] = c
    
brand_model_series = pd.Series(model_brand,name = 'model')
brand_model_count_series = pd.Series(model_brand_count,name = 'count')

df = pd.concat([brand_model_series, brand_model_count_series], axis=1)

In [29]:
df.sort_values(by='count', ascending=False)

,model,count
volkswagen,golf,3437
bmw,3er,2519
opel,corsa,1266
audi,a4,1199
mercedes_benz,c_klasse,1122
ford,focus,715
peugeot,2_reihe,579
renault,twingo,488
fiat,punto,321
seat,ibiza,303


Volkswagen Golf is the most occuring combination of brand and model.


In [30]:
#creating new dataset with only brand = volkswagen & model = golf
vw_golf = (autos.loc[(
    autos['brand']=='volkswagen')&
    (autos['model']=='golf')]
    )

vw_golf['gearbox'].value_counts().head(5)

manuell      2894
automatik     419
Name: gearbox, dtype: int64

Most of the cars left in the dataset have a manual transmission.

Let's see if the gearbox model have considerable effect on price

In [31]:
print(color.BOLD + 'manuell avg price : '+ color.RED +'{:,.2f}'.format(vw_golf.loc[vw_golf.gearbox == 'manuell' , 'price'].mean())+ color.END)
print(color.BOLD + 'automatik avg price :'+ color.RED +'{:,.2f}'.format(vw_golf.loc[vw_golf.gearbox == 'automatik' , 'price'].mean()))

manuell avg price : 4,852.35
automatik avg price :10,416.51


We can see that automatic models are significantly more expensive which seems reasonable of course

In [32]:
vw_golf['unrepaired_damage'].value_counts()

nein    2605
ja       235
Name: unrepaired_damage, dtype: int64

Although most of the cars don't have unrepaired damage, we can examine in what extent this factor change the price

In [33]:
print(color.BOLD + 'with unrepaired_damage : ' + color.RED + '{:,.2f}'.format(vw_golf.loc[vw_golf.unrepaired_damage == 'ja' , 'price'].mean())+color.END)
print(color.BOLD + 'without unrepaired_damage : ' + color.RED + '{:,.2f}'.format(vw_golf.loc[vw_golf.unrepaired_damage == 'nein' , 'price'].mean()))

with unrepaired_damage : 2,327.21
without unrepaired_damage : 6,250.50


So we can say that an unrepaired damage can reduce the golf model price almost 4000$ in average(which is more than half of the price)